## Google places API

En este codigo se obtienen todas la informacion relevante (ubicacion, puntuacion, reseñas, etc) de los restaurantes de NY para su posterior analisis. 

Se emplea una tecnica de division hexagonal y paginacion para extraer todos los posibles resultados. Finalmente se eliminan los duplicados generados por solapamiento en la busqueda

### Librerias necesarias
pandas requests python-dotenv

In [5]:
import requests
import pandas as pd
from tqdm import tqdm
import time
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()
API_KEY = os.getenv('../.env/GOOGLE_API_KEY')
